# Univariate logistic regression with leave one out cross validation 

Outputs significant features with the data size (n1= # of positive, n2=# of negative)

Also outputs the entire dataframe of features tested, sorted by significance. 

group 1 is 'positive' (binary 1), and group 2 is 'negative' (binary 0), p-value is Mann-Whitney U test, and bh p-value is the multiple comparisons corrected p-value. 

For individual significant parameters, AUC (95%CI), pvalue, Sensitivity (SN), specificity (SP), and the optimal probability threshold based on the J-statististic is reported. (less bias = closer to 0.50)

Note that the actual cut off value for a feature is not provided. This is because the cutoff values are different per each cross-validated loop. This means that the cut-off value may be different for every training set, while the probability is from the entire AUC that is made up of N different training sets on each of the N testing sets.

Mira 12/11/2024

In [1]:
OUTDATED_IGNORE=1
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
%matplotlib inline
import csv

import scipy.optimize as op
import scipy.stats
from scipy.optimize import curve_fit
import scipy.io
from scipy.stats import rice

import random
import pickle
import seaborn as sns

import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, wilcoxon,ttest_rel, pearsonr,shapiro,f_oneway, ranksums
from scipy.special import stdtr
import csv
import pandas as pd

from scipy.integrate import quad
import sys 
import os


from SomeUsefulFunctions import *

from scipy import special

from scipy.integrate import tplquad

pd.options.display.float_format = '{:.3f}'.format


import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
 

In [2]:
fileloc = '/Users/miraliu/Desktop/Data/4Dflow/PH_Summary.xlsx'
#Multiexp_Data = pd.read_excel(fileloc,sheet_name = 'Voxelwise_sortedfourpeaks_take2') ## take2 or not here
PH_Database = pd.read_excel(fileloc,sheet_name = 'Sheet2') ## take2 or not here


# first for "PH"

In [3]:
ParameterComparisonHead = ['parameter name', 'group 1 $\mu \pm \sigma$', 'group 2 $\mu \pm \sigma$', 'p-value']

df = PH_Database
df=df.drop(['HVPG', 'CSPH', 'Fibrosis Stage', 'GEVs','Binary Fibrosis'], axis=1)
df=df.dropna(subset='PH')

Pos = np.array(df.index[df['PH'] == 1])
Neg = np.array(df.index[df['PH'] == 0])
ParameterComparisons = CompareAB_manualrange_bh(df, Pos, Neg, ParameterComparisonHead, 1,45)

# ones with statistical significance
display(ParameterComparisons.loc[ParameterComparisons['p-value'] <.05])
display(ParameterComparisons.sort_values('p-value'))

print('\n------------------------ Individual Significant Parameters')
AllFeatures = ParameterComparisons['parameter name'].loc[ParameterComparisons['p-value'] <=0.05]
for index, row in AllFeatures.items():
    df_clean = df.dropna(subset=row)
    #print(row)
    auc, ci, sensitivity, specificity, Optimal_Prob,pval = Run_Logistic_Regression_delong_youden_noplot_INDIVIDUAL_loocv(df_clean,row,'PH','balanced')
    #if ci[0]>0.5:
    print(f'{row}: \nAUC={auc:.2f}, [{ci[0]:.2f},{ci[1]:.2f}], p={pval:.3f} \nSN={sensitivity:.2f}, SP={specificity:.2f}, Jstat Optimal threshold={Optimal_Prob:.3f}\n---------------')



SV Avg Th-Plane Vel    n1=11 n2=21
RHV Avg Th-Plane Vel    n1=11 n2=20
PV Peak Th-Plane Vel    n1=12 n2=21
SV Peak Th-Plane Vel    n1=11 n2=21
RHV Peak Th-Plane Vel    n1=11 n2=20
RHV Avg Net Flow    n1=11 n2=20
ScAo Avg Area    n1=15 n2=23


,parameter name,group 1 $\mu \pm \sigma$,group 2 $\mu \pm \sigma$,p-value,bh p-value
30,RHV Avg Net Flow,2.95± 1.63,5.46± 2.11,0.004,0.138
33,ScAo Avg Area,330.19± 46.65,273.21± 80.19,0.006,0.138
19,RHV Peak Th-Plane Vel,12.91± 5.64,20.56± 8.51,0.012,0.173
15,PV Peak Th-Plane Vel,17.01± 3.33,21.02± 6.11,0.033,0.256
8,RHV Avg Th-Plane Vel,5.7± 3.72,7.4± 2.78,0.043,0.256
6,SV Avg Th-Plane Vel,5.82± 1.88,8.04± 2.97,0.045,0.256
17,SV Peak Th-Plane Vel,12.23± 4.23,16.13± 5.24,0.045,0.256


,parameter name,group 1 $\mu \pm \sigma$,group 2 $\mu \pm \sigma$,p-value,bh p-value
30,RHV Avg Net Flow,2.95± 1.63,5.46± 2.11,0.004,0.138
33,ScAo Avg Area,330.19± 46.65,273.21± 80.19,0.006,0.138
19,RHV Peak Th-Plane Vel,12.91± 5.64,20.56± 8.51,0.012,0.173
15,PV Peak Th-Plane Vel,17.01± 3.33,21.02± 6.11,0.033,0.256
8,RHV Avg Th-Plane Vel,5.7± 3.72,7.4± 2.78,0.043,0.256
6,SV Avg Th-Plane Vel,5.82± 1.88,8.04± 2.97,0.045,0.256
17,SV Peak Th-Plane Vel,12.23± 4.23,16.13± 5.24,0.045,0.256
10,IVC_AL Avg Th-Plane Vel,23.3± 9.61,16.43± 5.74,0.056,0.256
11,ScAo Peak Th-Plane Vel,63.0± 19.46,74.55± 20.91,0.058,0.256
12,CT Peak Th-Plane Vel,52.46± 13.28,41.42± 13.52,0.059,0.256



------------------------ Individual Significant Parameters
RHV Avg Net Flow: 
AUC=0.78, [0.61,0.95], p=0.001 
SN=0.64, SP=0.75, Jstat Optimal threshold=0.517
---------------
ScAo Avg Area: 
AUC=0.74, [0.58,0.91], p=0.004 
SN=0.73, SP=0.74, Jstat Optimal threshold=0.539
---------------
RHV Peak Th-Plane Vel: 
AUC=0.74, [0.54,0.95], p=0.021 
SN=0.73, SP=0.75, Jstat Optimal threshold=0.579
---------------
PV Peak Th-Plane Vel: 
AUC=0.69, [0.49,0.88], p=0.060 
SN=0.58, SP=0.71, Jstat Optimal threshold=0.567
---------------
RHV Avg Th-Plane Vel: 
AUC=0.66, [0.44,0.88], p=0.151 
SN=0.73, SP=0.65, Jstat Optimal threshold=0.507
---------------
SV Avg Th-Plane Vel: 
AUC=0.69, [0.50,0.88], p=0.049 
SN=0.91, SP=0.43, Jstat Optimal threshold=0.344
---------------
SV Peak Th-Plane Vel: 
AUC=0.67, [0.47,0.87], p=0.091 
SN=0.55, SP=0.76, Jstat Optimal threshold=0.559
---------------


# for CSPH

In [4]:
ParameterComparisonHead = ['parameter name', 'group 1 $\mu \pm \sigma$', 'group 2 $\mu \pm \sigma$', 'p-value']

df = PH_Database
df=df.drop(['HVPG', 'PH', 'Fibrosis Stage', 'GEVs','Binary Fibrosis'], axis=1)
df=df.dropna(subset='CSPH')

Pos = np.array(df.index[df['CSPH'] == 1])
Neg = np.array(df.index[df['CSPH'] == 0])
ParameterComparisons = CompareAB_manualrange_bh(df, Pos, Neg, ParameterComparisonHead, 1,45)

# ones with statistical significance
display(ParameterComparisons.loc[ParameterComparisons['p-value'] <.05])
display(ParameterComparisons.sort_values('p-value'))

print('\n------------------------ Individual Significant Parameters')
AllFeatures = ParameterComparisons['parameter name'].loc[ParameterComparisons['p-value'] <=0.05]
for index, row in AllFeatures.items():
    df_clean = df.dropna(subset=row)
    #print(row)
    auc, ci, sensitivity, specificity, Optimal_Prob,pval = Run_Logistic_Regression_delong_youden_noplot_INDIVIDUAL_loocv(df_clean,row,'CSPH','balanced')
    #if ci[0]>0.5:
    print(f'{row}: \nAUC={auc:.2f}, [{ci[0]:.2f},{ci[1]:.2f}], p={pval:.3f} \nSN={sensitivity:.2f}, SP={specificity:.2f}, Jstat thresh={Optimal_Prob:.3f}\n---------------')


HA Avg Th-Plane Vel    n1=5 n2=14
MHV Avg Th-Plane Vel    n1=2 n2=14
CT Peak Th-Plane Vel    n1=5 n2=30
HA Peak Th-Plane Vel    n1=5 n2=14
ScAo Avg Area    n1=6 n2=32
PV Avg Area    n1=4 n2=29


,parameter name,group 1 $\mu \pm \sigma$,group 2 $\mu \pm \sigma$,p-value,bh p-value
33,ScAo Avg Area,358.86± 44.19,283.86± 72.87,0.008,0.286
2,HA Avg Th-Plane Vel,14.11± 6.08,8.4± 2.94,0.016,0.286
12,CT Peak Th-Plane Vel,61.46± 12.17,43.23± 13.12,0.027,0.286
13,HA Peak Th-Plane Vel,38.28± 15.71,21.95± 8.1,0.033,0.286
37,PV Avg Area,138.41± 28.93,93.8± 28.95,0.036,0.286
7,MHV Avg Th-Plane Vel,12.74± 1.61,7.12± 2.84,0.039,0.286


,parameter name,group 1 $\mu \pm \sigma$,group 2 $\mu \pm \sigma$,p-value,bh p-value
33,ScAo Avg Area,358.86± 44.19,283.86± 72.87,0.008,0.286
2,HA Avg Th-Plane Vel,14.11± 6.08,8.4± 2.94,0.016,0.286
12,CT Peak Th-Plane Vel,61.46± 12.17,43.23± 13.12,0.027,0.286
13,HA Peak Th-Plane Vel,38.28± 15.71,21.95± 8.1,0.033,0.286
37,PV Avg Area,138.41± 28.93,93.8± 28.95,0.036,0.286
7,MHV Avg Th-Plane Vel,12.74± 1.61,7.12± 2.84,0.039,0.286
24,HA Avg Net Flow,6.34± 2.98,3.7± 1.86,0.064,0.394
22,ScAo Avg Net Flow,54.49± 10.54,42.46± 16.76,0.072,0.394
23,CT Avg Net Flow,10.03± 2.84,7.07± 3.36,0.081,0.397
10,IVC_AL Avg Th-Plane Vel,23.58± 7.57,18.04± 7.84,0.114,0.470



------------------------ Individual Significant Parameters
ScAo Avg Area: 
AUC=0.78, [0.51,1.00], p=0.043 
SN=0.67, SP=0.78, Jstat thresh=0.554
---------------
HA Avg Th-Plane Vel: 
AUC=0.79, [0.55,1.00], p=0.019 
SN=0.60, SP=0.79, Jstat thresh=0.415
---------------
CT Peak Th-Plane Vel: 
AUC=0.73, [0.38,1.00], p=0.200 
SN=0.40, SP=0.97, Jstat thresh=0.759
---------------
HA Peak Th-Plane Vel: 
AUC=0.74, [0.44,1.00], p=0.117 
SN=0.60, SP=0.71, Jstat thresh=0.434
---------------
PV Avg Area: 
AUC=0.78, [0.55,1.00], p=0.016 
SN=0.75, SP=0.55, Jstat thresh=0.327
---------------
MHV Avg Th-Plane Vel: 
AUC=0.89, [0.66,1.00], p=0.001 
SN=0.50, SP=0.79, Jstat thresh=0.244
---------------


# for GEVs

Multiple comparisons correction not possible because some features don't even have one case in either group

In [5]:
ParameterComparisonHead = ['parameter name', 'group 1 $\mu \pm \sigma$', 'group 2 $\mu \pm \sigma$', 'p-value']

df = PH_Database
df=df.drop(['HVPG', 'PH', 'Fibrosis Stage', 'CSPH', 'Binary Fibrosis'], axis=1)
df=df.dropna(subset='GEVs')

Pos = np.array(df.index[df['GEVs'] == 1])
Neg = np.array(df.index[df['GEVs'] == 0])
ParameterComparisons = CompareAB_manualrange_bh(df, Pos, Neg, ParameterComparisonHead, 1,45)

# ones with statistical significance
display(ParameterComparisons.loc[ParameterComparisons['p-value'] <.05])
display(ParameterComparisons.sort_values('p-value'))

print('\n------------------------ Individual Significant Parameters')
AllFeatures = ParameterComparisons['parameter name'].loc[ParameterComparisons['p-value'] <=0.05]
for index, row in AllFeatures.items():
    df_clean = df.dropna(subset=row)
    #print(row)
    auc, ci, sensitivity, specificity, Optimal_Prob,pval = Run_Logistic_Regression_delong_youden_noplot_INDIVIDUAL_loocv(df_clean,row,'GEVs','balanced')
    #if ci[0]>0.5:
    print(f'{row}: \nAUC={auc:.2f}, [{ci[0]:.2f},{ci[1]:.2f}], p={pval:.3f} \nSN={sensitivity:.2f}, SP={specificity:.2f}, Jstat Optimal threshold={Optimal_Prob:.3f}\n---------------')


CT Avg Net Flow    n1=8 n2=14
SA Avg Net Flow    n1=8 n2=13
SA Avg Area    n1=8 n2=13
IVC_AL Avg Area    n1=7 n2=11


,parameter name,group 1 $\mu \pm \sigma$,group 2 $\mu \pm \sigma$,p-value,bh p-value
25,SA Avg Net Flow,5.93± 1.99,3.49± 1.8,0.025,NaN
36,SA Avg Area,48.65± 23.12,33.13± 10.72,0.025,NaN
43,IVC_AL Avg Area,250.22± 103.59,145.93± 67.67,0.026,NaN
23,CT Avg Net Flow,9.86± 3.76,6.76± 2.88,0.034,NaN


,parameter name,group 1 $\mu \pm \sigma$,group 2 $\mu \pm \sigma$,p-value,bh p-value
25,SA Avg Net Flow,5.93± 1.99,3.49± 1.8,0.025,NaN
36,SA Avg Area,48.65± 23.12,33.13± 10.72,0.025,NaN
43,IVC_AL Avg Area,250.22± 103.59,145.93± 67.67,0.026,NaN
23,CT Avg Net Flow,9.86± 3.76,6.76± 2.88,0.034,NaN
12,CT Peak Th-Plane Vel,55.98± 13.11,44.09± 13.44,0.065,NaN
35,HA Avg Area,55.54± 12.16,39.67± 11.14,0.088,NaN
32,IVC_AL Avg Net Flow,37.71± 11.21,28.51± 11.45,0.113,NaN
16,SMV Peak Th-Plane Vel,8.59± 0.58,15.06± 7.75,0.117,NaN
5,SMV Avg Th-Plane Vel,3.5± 0.7,6.98± 2.12,0.117,NaN
38,SMV Avg Area,109.96± 32.79,65.23± 19.34,0.117,NaN



------------------------ Individual Significant Parameters
SA Avg Net Flow: 
AUC=0.76, [0.53,0.99], p=0.029 
SN=0.75, SP=0.62, Jstat Optimal threshold=0.319
---------------
SA Avg Area: 
AUC=0.77, [0.55,0.98], p=0.014 
SN=0.75, SP=0.69, Jstat Optimal threshold=0.421
---------------
IVC_AL Avg Area: 
AUC=0.77, [0.53,1.00], p=0.026 
SN=0.43, SP=0.91, Jstat Optimal threshold=0.721
---------------
CT Avg Net Flow: 
AUC=0.72, [0.47,0.98], p=0.089 
SN=0.50, SP=0.86, Jstat Optimal threshold=0.643
---------------


# for Fibrosis <4

Multiple comparisons correction not possible because some features don't even have one case in either group

In [6]:
ParameterComparisonHead = ['parameter name', 'group 1 $\mu \pm \sigma$', 'group 2 $\mu \pm \sigma$', 'p-value']

df = PH_Database
df=df.drop(['HVPG', 'PH', 'GEVs', 'CSPH','Fibrosis Stage'], axis=1)
df=df.dropna(subset='Binary Fibrosis')

BinaryYes = np.array(df.index[df['Binary Fibrosis'] ==True])
BinaryNo = np.array(df.index[df['Binary Fibrosis'] == False])
ParameterComparisons = CompareAB_manualrange_bh(df, BinaryYes, BinaryNo, ParameterComparisonHead, 1,45)

# ones with statistical significance
display(ParameterComparisons.loc[ParameterComparisons['p-value'] <.05])
display(ParameterComparisons.sort_values('p-value'))

print('\n------------------------ Individual Significant Parameters')
AllFeatures = ParameterComparisons['parameter name'].loc[ParameterComparisons['p-value'] <=0.05]
for index, row in AllFeatures.items():
    df_clean = df.dropna(subset=row)
    #print(row)
    auc, ci, sensitivity, specificity, Optimal_Prob,pval = Run_Logistic_Regression_delong_youden_noplot_INDIVIDUAL_loocv(df_clean,row,'Binary Fibrosis','balanced')
    #if ci[0]>0.5:
    print(f'{row}: \nAUC={auc:.2f}, [{ci[0]:.2f},{ci[1]:.2f}], p={pval:.3f} \nSN={sensitivity:.2f}, SP={specificity:.2f}, Jstat Optimal threshold={Optimal_Prob:.3f}\n---------------')


RHV Avg Th-Plane Vel    n1=5 n2=26
RHV Peak Th-Plane Vel    n1=5 n2=26
IVC_AL Peak Th-Plane Vel    n1=6 n2=28
ScAo Avg Net Flow    n1=9 n2=29
ScAo Avg Area    n1=9 n2=29
CT Avg Area    n1=9 n2=26


,parameter name,group 1 $\mu \pm \sigma$,group 2 $\mu \pm \sigma$,p-value,bh p-value
33,ScAo Avg Area,347.17± 19.2,279.73± 77.79,0.002,NaN
21,IVC_AL Peak Th-Plane Vel,72.39± 12.22,48.25± 15.97,0.003,NaN
22,ScAo Avg Net Flow,57.4± 18.17,40.31± 13.65,0.015,NaN
19,RHV Peak Th-Plane Vel,10.46± 3.44,19.26± 8.39,0.016,NaN
34,CT Avg Area,76.41± 39.25,60.59± 49.04,0.024,NaN
8,RHV Avg Th-Plane Vel,4.52± 1.3,7.23± 3.33,0.036,NaN


,parameter name,group 1 $\mu \pm \sigma$,group 2 $\mu \pm \sigma$,p-value,bh p-value
33,ScAo Avg Area,347.17± 19.2,279.73± 77.79,0.002,NaN
21,IVC_AL Peak Th-Plane Vel,72.39± 12.22,48.25± 15.97,0.003,NaN
22,ScAo Avg Net Flow,57.4± 18.17,40.31± 13.65,0.015,NaN
19,RHV Peak Th-Plane Vel,10.46± 3.44,19.26± 8.39,0.016,NaN
34,CT Avg Area,76.41± 39.25,60.59± 49.04,0.024,NaN
8,RHV Avg Th-Plane Vel,4.52± 1.3,7.23± 3.33,0.036,NaN
24,HA Avg Net Flow,5.82± 1.82,4.02± 2.52,0.057,NaN
23,CT Avg Net Flow,9.24± 2.78,6.89± 3.45,0.070,NaN
35,HA Avg Area,54.19± 11.14,41.38± 12.35,0.072,NaN
17,SV Peak Th-Plane Vel,11.68± 3.69,15.51± 5.3,0.082,NaN



------------------------ Individual Significant Parameters
ScAo Avg Area: 
AUC=0.83, [0.70,0.96], p=0.000 
SN=0.89, SP=0.69, Jstat Optimal threshold=0.459
---------------
IVC_AL Peak Th-Plane Vel: 
AUC=0.86, [0.71,1.00], p=0.000 
SN=0.67, SP=0.89, Jstat Optimal threshold=0.578
---------------
ScAo Avg Net Flow: 
AUC=0.72, [0.48,0.96], p=0.068 
SN=0.44, SP=0.97, Jstat Optimal threshold=0.713
---------------
RHV Peak Th-Plane Vel: 
AUC=0.75, [0.43,1.00], p=0.127 
SN=0.60, SP=0.85, Jstat Optimal threshold=0.674
---------------
CT Avg Area: 
AUC=0.68, [0.44,0.91], p=0.134 
SN=0.67, SP=0.81, Jstat Optimal threshold=0.483
---------------
RHV Avg Th-Plane Vel: 
AUC=0.73, [0.45,1.00], p=0.104 
SN=0.60, SP=0.69, Jstat Optimal threshold=0.639
---------------
